In [2]:
pip install pyttsx3


   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtypes]
   ------------- -------------------------- 1/3 [comtyp

In [1]:
import pyttsx3

engine = pyttsx3.init()  # object creation

# RATE
rate = engine.getProperty("rate")  # getting details of current speaking rate
print(rate)  # printing current voice rate
engine.setProperty("rate", 145)  # setting up new voice rate

# VOLUME
volume = engine.getProperty(
    "volume"
)  # getting to know current volume level (min=0 and max=1)
print(volume)  # printing current volume level
engine.setProperty("volume", 1.0)  # setting up volume level  between 0 and 1

engine.say("Hello, how is it going!")
engine.say("My current speaking rate is " + str(rate))
engine.runAndWait()
engine.stop()

200
1.0


In [ ]:
engine = pyttsx3.init()
rate = engine.getProperty("rate")
engine.setProperty("rate", 145)
volume = engine.getProperty(
    "volume"
)  # getting to know current volume level (min=0 and max=1)
engine.setProperty("volume", 1.0)


def text_to_speech(engine: pyttsx3, text: str, volume: float):
    engine.setProperty("volume", volume)
    engine.say(text)
    engine.runAndWait()
    engine.stop()

In [4]:
pip install g29py

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for hid: filename=hid-1.0.4-py3-none-any.whl size=3857 sha256=d9026857a58a905877bb774341ae3ab65c1185cd5742993414f17aa0e804a1d3
  Stored in directory: c:\users\pasupuleti\appdata\local\pip\cache\wheels\5b\6d\94\84b95e56dfa4388ccb00d936a57cfcf6e93815683f5cd6303d
Successfully built hid

   ---------------------------------------- 2/2 [g29py]

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install hidapi

  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)

   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   ---------------------------------------- 0/2 [setuptools]
   -----------------

In [3]:
pip install hid

Note: you may need to restart the kernel to use updated packages.


In [2]:
import time

import logitech_steering_wheel as lsw
import pygetwindow as gw

FileNotFoundError: Could not find module 'c:\Users\pasupuleti\Desktop\carla-drowsiness-detection\.venv\Lib\site-packages\logitech_steering_wheel\lib\x64\LogitechSteeringWheelEnginesWrapper.dll' (or one of its dependencies). Try using the full path with constructor syntax.